In [10]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import sys, os
sys.path.append("../subjects/")
import time, random
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, accuracy_score
from fairlearn.metrics import equalized_odds_difference
from Search_utils import causal_dataset_law
from sklearn.feature_selection import SelectKBest, SelectFpr,SelectPercentile 

import matplotlib.pyplot as plt
import itertools
org_data = pd.read_csv('../subjects/datasets/law_pp.csv')
coef_data = pd.read_csv('../Results/LogisticRegression/law/sex/law_coef.csv')

#coef_data['ry'] =coef_data['ry'] -1.9

    
#coef_data['sy']=coef_data['sy'] -0.5
coef_list  = coef_data.columns[8:]
    
coef_mean = coef_data[coef_data.columns[8:]].mean()
non_sensitive = ['race', 'LSAT', 'UGPA', 'label']


sensitive_param = 'sex'

transformer ='SelectKBest'#,'SelectFpr','SelectPercentile']:
feat_name_list = []
#coef_data['sy']=coef_data['sy'] -2.5

analysis = 'nonsensitive'
drop_analysis_result_SelectKBest=[]
drop_analysis_result_SelectFpr=[]
drop_analysis_result_SelectPercentile=[]
drop_analysis_result_sens= []
stp_range =[i/10 for i in range(-40,40,4)]
results ={}
for coef in coef_list:
    #coef_data[coef] =7
    if '0' in coef or 'sigma' in coef:continue
        
    for step_size in stp_range: 
        if step_size==0:
            continue

        eod_d =[]
        acc_d =[]
        f1_d=[]

        
        
        #EOD_N.append(EOD_None)
        status, df, bal, EOD_None, acc_None, f1_None = causal_dataset_law(org_data, coef_data,
                                                        coef_list, coef, step_size  ,False, sensitive_param , default_acc=None  , 
                                                        default_f1=None, random_state= 0,  drop_feat=None)
        if status==False:
            continue
            
        
        #EOD_N=[]
        for exp in range(10):
            

            A_new = df[sensitive_param].to_numpy()
            X_new = df[df.columns[:-1]].drop(columns=[sensitive_param])
            Y_new = df[df.columns[-1]]
            if transformer == 'SelectKBest' and analysis == 'nonsensitive':
                Kbest = SelectKBest( k=len(non_sensitive)-1)
                X_new = Kbest.fit_transform(X_new, Y_new)

            elif transformer == 'SelectFpr'and analysis == 'nonsensitive':
                sfpr =  SelectFpr(alpha=0.01)
                X_new = sfpr.fit_transform(X_new, Y_new)

            elif transformer == 'SelectPercentile'and analysis == 'nonsensitive':
                percentile =  SelectPercentile(percentile=10)
                X_new = percentile.fit_transform(X_new, Y_new)

            if analysis == 'nonsensitive':
                X_new = np.hstack((X_new,A_new.reshape(-1,1)))







            X_train_new, X_test_new, y_train_new, y_test_new, A_train_new, A_test_new = train_test_split(X_new, 
                                                                                                         Y_new,
                                                                                                         A_new, 
                                                                                                       random_state=None,
                                                                                                         stratify=Y_new)

            model = LogisticRegression()
            model.fit(X_train_new,y_train_new)
            preds_new = model.predict(X_test_new)
            acc_temp = accuracy_score(y_test_new,preds_new)
            f1_temp = f1_score(y_test_new,preds_new)
            EOD = round(equalized_odds_difference(y_test_new, preds_new, sensitive_features=A_test_new ),3)
            print(EOD,EOD_None)
            acc_diff = round(acc_temp - acc_None,2)
            f1_diff = round(f1_temp- f1_None,2)
            EOD_diff = EOD - EOD_None
            #print(EOD_None, EOD)
            eod_d.append(EOD_diff)
            acc_d.append(acc_diff)
            f1_d.append(f1_diff)
            #print(coef, step_size,np.mean(eod_d),np.mean(acc_d),np.mean(f1_d))
        if transformer == 'SelectKBest' and analysis == 'nonsensitive':
            drop_analysis_result_SelectKBest.append([coef, step_size, eod_d, acc_d, f1_d])
        elif transformer == 'SelectFpr' and analysis == 'nonsensitive':
            drop_analysis_result_SelectFpr.append([coef, step_size, eod_d, acc_d, f1_d])
        elif transformer == 'SelectPercentile' and analysis == 'nonsensitive':
            drop_analysis_result_SelectPercentile.append([coef, step_size, eod_d, acc_d, f1_d])

        if analysis == 'sensitive':
            drop_analysis_result_sens.append([coef, step_size, eod_d, acc_d, f1_d])
        #print(coef, step_size,np.mean(eod))
        print(coef, step_size,np.mean(eod_d),np.mean(acc_d),np.mean(f1_d))
if transformer == 'SelectKBest':
        np.save('../Results/LogisticRegression/law/st_test/RQ2/SelectKBest.npy',
                  drop_analysis_result_SelectKBest)
elif transformer == 'SelectFpr':
            np.save('../Results/LogisticRegression/law/st_test/RQ2/SelectFpr.npy',
                  drop_analysis_result_SelectFpr)
elif transformer == 'SelectPercentile':
            np.save('../Results/LogisticRegression/law/st_test/RQ2/SelectPercentile.npy',
                  drop_analysis_result_SelectPercentile)
if analysis == 'sensitive': 
           np.save('../Results/LogisticRegression/law/st_test/RQ2/Sensitive.npy',
                  drop_analysis_result_sens)

0.013 0.021
0.03 0.021
0.004 0.021
0.03 0.021
0.013 0.021
0.019 0.021
0.022 0.021
0.018 0.021
0.008 0.021
0.021 0.021
rg -4.0 -0.0032000000000000015 0.0 0.0
0.026 0.021
0.021 0.021
0.021 0.021
0.012 0.021
0.022 0.021
0.025 0.021
0.013 0.021
0.016 0.021
0.002 0.021
0.03 0.021
rg -3.6 -0.0022000000000000014 0.0 0.0
0.016 0.021
0.028 0.021
0.012 0.021
0.021 0.021
0.027 0.021
0.006 0.021
0.009 0.021
0.014 0.021
0.013 0.021
0.033 0.021
rg -3.2 -0.0031000000000000003 0.0 0.0
0.018 0.021
0.025 0.021
0.043 0.021
0.031 0.021
0.035 0.021
0.015 0.021
0.018 0.021
0.017 0.021
0.017 0.021
0.017 0.021
rg -2.8 0.002599999999999999 0.0 0.0
0.02 0.021
0.014 0.021
0.004 0.021
0.013 0.021
0.017 0.021
0.012 0.021
0.004 0.021
0.022 0.021
0.025 0.021
0.026 0.021
rg -2.4 -0.005300000000000001 0.0 0.0
0.029 0.021
0.018 0.021
0.008 0.021
0.016 0.021
0.025 0.021
0.022 0.021
0.028 0.021
0.016 0.021
0.015 0.021
0.017 0.021
rg -2.0 -0.0016000000000000012 0.0 0.0
0.013 0.021
0.024 0.021
0.025 0.021
0.023 0.021
0.023

0.036 0.021
0.009 0.021
0.012 0.021
0.025 0.021
sg -3.6 0.0025999999999999986 0.0 0.0
0.009 0.021
0.002 0.021
0.012 0.021
0.017 0.021
0.017 0.021
0.004 0.021
0.02 0.021
0.012 0.021
0.03 0.021
0.008 0.021
sg -3.2 -0.0079 0.0 0.0
0.021 0.021
0.016 0.021
0.013 0.021
0.028 0.021
0.021 0.021
0.026 0.021
0.033 0.021
0.028 0.021
0.022 0.021
0.026 0.021
sg -2.8 0.002399999999999999 0.0 0.0
0.024 0.021
0.023 0.021
0.017 0.021
0.021 0.021
0.022 0.021
0.009 0.021
0.018 0.021
0.021 0.021
0.016 0.021
0.023 0.021
sg -2.4 -0.0016000000000000012 0.0 0.0
0.007 0.021
0.037 0.021
0.026 0.021
0.026 0.021
0.018 0.021
0.027 0.021
0.026 0.021
0.023 0.021
0.024 0.021
0.009 0.021
sg -2.0 0.0012999999999999978 0.0 0.0
0.029 0.021
0.016 0.021
0.016 0.021
0.017 0.021
0.017 0.021
0.032 0.021
0.009 0.021
0.004 0.021
0.017 0.021
0.014 0.021
sg -1.6 -0.0039000000000000007 0.0 0.0
0.013 0.021
0.034 0.021
0.021 0.021
0.034 0.021
0.013 0.021
0.021 0.021
0.029 0.021
0.005 0.021
0.024 0.021
0.018 0.021
sg -1.2 0.000199999

0.017 0.013
0.009 0.013
0.012 0.013
0.017 0.013
0.001 0.013
0.008 0.013
0.013 0.013
0.017 0.013
sy 2.8 -0.0002999999999999992 0.0 0.0
0.004 0.017
0.009 0.017
0.013 0.017
0.008 0.017
0.021 0.017
0.002 0.017
0.017 0.017
0.008 0.017
0.004 0.017
0.001 0.017
sy 3.2 -0.008300000000000002 0.0 0.0
0.008 0.008
0.017 0.008
0.012 0.008
0.012 0.008
0.013 0.008
0.021 0.008
0.034 0.008
0.008 0.008
0.028 0.008
0.02 0.008
sy 3.6 0.0093 0.0 0.0


In [11]:
np.mean(np.load('../Results/LogisticRegression/law/st_test/RQ2/SelectFpr_high_final.npy',allow_pickle=True)[0][2])

-0.01555

In [40]:
np.np.mean(drop_analysis_result_SelectFpr[0][2])

-0.22329000000000004

In [ ]:
ry -1.9 -0.1589 0.0 -0.113
ry -1.9 -0.15607 0.0 -0.10950000000000001